In [ ]:
import sys
import cv2
import torch
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap, QFont
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QHBoxLayout, QPushButton, QWidget, QFrame
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image

# Charger le modèle de classification
processor = AutoImageProcessor.from_pretrained("dima806/asl_alphabet_image_detection")
model = AutoModelForImageClassification.from_pretrained("dima806/asl_alphabet_image_detection")

def preprocess_frame(frame):
    """Convertit un cadre de la caméra en image PIL et le prétraite pour la classification"""
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)
    inputs = processor(images=pil_image, return_tensors="pt")
    return inputs

def classify_asl_frame(frame):
    """Classifie un cadre pour détecter la lettre en langage des signes"""
    inputs = preprocess_frame(frame)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    predicted_label = model.config.id2label[predicted_class_idx]
    confidence = torch.softmax(logits, dim=-1)[0][predicted_class_idx].item()
    return predicted_label, confidence * 100

class ASLApp(QWidget):
    def __init__(self):
        super().__init__()

        # Configuration de la fenêtre
        self.setWindowTitle("Détection de la Langue des Signes")
        self.setGeometry(100, 100, 900, 700)
        self.setStyleSheet("background-color: #1a1a1a; color: white;")

        # Layout principal
        self.layout = QVBoxLayout()

        # Titre
        self.title_label = QLabel("D\u00e9tection de la Langue des Signes")
        self.title_label.setAlignment(Qt.AlignCenter)
        self.title_label.setFont(QFont("Arial", 24, QFont.Bold))
        self.title_label.setStyleSheet("color: white;")
        self.layout.addWidget(self.title_label)

        # Layout pour la vidéo et les résultats
        self.video_layout = QHBoxLayout()

        # Affichage de la vidéo
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setStyleSheet("background-color: #333;")
        self.video_layout.addWidget(self.video_label)

        # Résultats
        self.result_frame = QFrame()
        self.result_frame.setStyleSheet("background-color: #2e2e2e; border: 1px solid #555;")
        self.result_layout = QVBoxLayout()

        self.character_label = QLabel("Caract\u00e8re :")
        self.character_label.setFont(QFont("Arial", 18))
        self.character_label.setAlignment(Qt.AlignCenter)
        self.character_label.setStyleSheet("color: white;")
        self.result_layout.addWidget(self.character_label)

        self.word_label = QLabel("Mot :")
        self.word_label.setFont(QFont("Arial", 18))
        self.word_label.setAlignment(Qt.AlignCenter)
        self.word_label.setStyleSheet("color: white;")
        self.result_layout.addWidget(self.word_label)

        self.result_frame.setLayout(self.result_layout)
        self.video_layout.addWidget(self.result_frame)

        self.layout.addLayout(self.video_layout)

        # Boutons de contrôle
        self.buttons_layout = QHBoxLayout()

        self.start_button = QPushButton("D\u00e9marrer")
        self.start_button.setStyleSheet("background-color: #007bff; color: white;")
        self.start_button.clicked.connect(self.start_detection)
        self.buttons_layout.addWidget(self.start_button)

        self.stop_button = QPushButton("Arr\u00eater")
        self.stop_button.setStyleSheet("background-color: #dc3545; color: white;")
        self.stop_button.clicked.connect(self.stop_detection)
        self.buttons_layout.addWidget(self.stop_button)

        self.quit_button = QPushButton("Quitter")
        self.quit_button.setStyleSheet("background-color: #6c757d; color: white;")
        self.quit_button.clicked.connect(self.close)
        self.buttons_layout.addWidget(self.quit_button)

        self.layout.addLayout(self.buttons_layout)

        # Configuration de la fenêtre
        self.setLayout(self.layout)

        # Timer pour mettre à jour la vidéo
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)

        # Capture vidéo
        self.cap = None

        # Variables pour le mot en cours
        self.current_word = ""
        self.frame_counter = 0
        self.last_letter = ""

    def start_detection(self):
        """Démarrer la détection"""
        if not self.cap:
            self.cap = cv2.VideoCapture(0)
        self.timer.start(300)

    def stop_detection(self):
        """Arrêter la détection"""
        if self.cap:
            self.timer.stop()
            self.cap.release()
            self.cap = None
        self.video_label.clear()
        self.character_label.setText("Caract\u00e8re :")
        self.word_label.setText("Mot :")
        self.current_word = ""
        self.frame_counter = 0

    def update_frame(self):
        """Mettre à jour le flux vidéo et effectuer la classification"""
        if self.cap and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                # Afficher la vidéo
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                h, w, ch = rgb_frame.shape
                bytes_per_line = ch * w
                qimg = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
                self.video_label.setPixmap(QPixmap.fromImage(qimg))

                # Effectuer la classification toutes les 20 images
                self.frame_counter += 1
                if self.frame_counter % 20 == 0:
                    try:
                        predicted_letter, confidence = classify_asl_frame(frame)
                        if predicted_letter != self.last_letter:
                            self.character_label.setText(f"Caract\u00e8re : {predicted_letter}")
                            self.current_word += predicted_letter
                            self.word_label.setText(f"Mot : {self.current_word}")
                            self.last_letter = predicted_letter
                    except Exception as e:
                        self.character_label.setText(f"Erreur : {e}")

    def closeEvent(self, event):
        """Libérer les ressources à la fermeture"""
        if self.cap:
            self.cap.release()
        super().closeEvent(event)

# Exécution de l'application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ASLApp()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
import cv2
import torch
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap, QFont
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QHBoxLayout, QPushButton, QWidget, QFrame
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Charger le modèle CNN
model = load_model("sign_language_cnn_model2.h5")
class_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
                 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 
                 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 
                 27: 'nothing', 28: 'space'}

def preprocess_frame(frame):
    """Convertit un cadre de la caméra en une image compatible avec le modèle CNN"""
    resized_frame = cv2.resize(frame, (64, 64))
    normalized_frame = resized_frame.astype('float32') / 255.0
    input_frame = img_to_array(normalized_frame)
    input_frame = np.expand_dims(input_frame, axis=0)  # Ajouter une dimension pour le batch
    return input_frame

def classify_asl_frame(frame):
    """Classifie un cadre pour détecter la lettre en langage des signes"""
    input_frame = preprocess_frame(frame)
    prediction = model.predict(input_frame)
    predicted_class_idx = np.argmax(prediction)
    confidence = np.max(prediction)
    predicted_label = class_mapping[predicted_class_idx]
    return predicted_label, confidence * 100

class ASLApp(QWidget):
    def __init__(self):
        super().__init__()

        # Configuration de la fenêtre
        self.setWindowTitle("Détection de la Langue des Signes")
        self.setGeometry(100, 100, 900, 700)
        self.setStyleSheet("background-color: #1a1a1a; color: white;")

        # Layout principal
        self.layout = QVBoxLayout()

        # Titre
        self.title_label = QLabel("D\u00e9tection de la Langue des Signes")
        self.title_label.setAlignment(Qt.AlignCenter)
        self.title_label.setFont(QFont("Arial", 24, QFont.Bold))
        self.title_label.setStyleSheet("color: white;")
        self.layout.addWidget(self.title_label)

        # Layout pour la vidéo et les résultats
        self.video_layout = QHBoxLayout()

        # Affichage de la vidéo
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setStyleSheet("background-color: #333;")
        self.video_layout.addWidget(self.video_label)

        # Résultats
        self.result_frame = QFrame()
        self.result_frame.setStyleSheet("background-color: #2e2e2e; border: 1px solid #555;")
        self.result_layout = QVBoxLayout()

        self.character_label = QLabel("Caract\u00e8re :")
        self.character_label.setFont(QFont("Arial", 18))
        self.character_label.setAlignment(Qt.AlignCenter)
        self.character_label.setStyleSheet("color: white;")
        self.result_layout.addWidget(self.character_label)

        self.word_label = QLabel("Mot :")
        self.word_label.setFont(QFont("Arial", 18))
        self.word_label.setAlignment(Qt.AlignCenter)
        self.word_label.setStyleSheet("color: white;")
        self.result_layout.addWidget(self.word_label)

        self.result_frame.setLayout(self.result_layout)
        self.video_layout.addWidget(self.result_frame)

        self.layout.addLayout(self.video_layout)

        # Boutons de contrôle
        self.buttons_layout = QHBoxLayout()

        self.start_button = QPushButton("D\u00e9marrer")
        self.start_button.setStyleSheet("background-color: #007bff; color: white;")
        self.start_button.clicked.connect(self.start_detection)
        self.buttons_layout.addWidget(self.start_button)

        self.stop_button = QPushButton("Arr\u00eater")
        self.stop_button.setStyleSheet("background-color: #dc3545; color: white;")
        self.stop_button.clicked.connect(self.stop_detection)
        self.buttons_layout.addWidget(self.stop_button)

        self.quit_button = QPushButton("Quitter")
        self.quit_button.setStyleSheet("background-color: #6c757d; color: white;")
        self.quit_button.clicked.connect(self.close)
        self.buttons_layout.addWidget(self.quit_button)

        self.layout.addLayout(self.buttons_layout)

        # Configuration de la fenêtre
        self.setLayout(self.layout)

        # Timer pour mettre à jour la vidéo
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)

        # Capture vidéo
        self.cap = None

        # Variables pour le mot en cours
        self.current_word = ""
        self.frame_counter = 0
        self.last_letter = ""

    def start_detection(self):
        """Démarrer la détection"""
        if not self.cap:
            self.cap = cv2.VideoCapture(0)
        self.timer.start(300)

    def stop_detection(self):
        """Arrêter la détection"""
        if self.cap:
            self.timer.stop()
            self.cap.release()
            self.cap = None
        self.video_label.clear()
        self.character_label.setText("Caract\u00e8re :")
        self.word_label.setText("Mot :")
        self.current_word = ""
        self.frame_counter = 0

    def update_frame(self):
        """Mettre à jour le flux vidéo et effectuer la classification"""
        if self.cap and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                # Afficher la vidéo
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                h, w, ch = rgb_frame.shape
                bytes_per_line = ch * w
                qimg = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
                self.video_label.setPixmap(QPixmap.fromImage(qimg))

                # Effectuer la classification toutes les 20 images
                self.frame_counter += 1
                if self.frame_counter % 20 == 0:
                    try:
                        predicted_letter, confidence = classify_asl_frame(frame)
                        if predicted_letter != self.last_letter:
                            self.character_label.setText(f"Caract\u00e8re : {predicted_letter} ({confidence:.2f}%)")
                            self.current_word += predicted_letter
                            self.word_label.setText(f"Mot : {self.current_word}")
                            self.last_letter = predicted_letter
                    except Exception as e:
                        self.character_label.setText(f"Erreur : {e}")

    def closeEvent(self, event):
        """Libérer les ressources à la fermeture"""
        if self.cap:
            self.cap.release()
        super().closeEvent(event)

# Exécution de l'application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ASLApp()
    window.show()
    sys.exit(app.exec_())
